# Replicating Jorion (2003)
Portfolio optimization with tracking error constraints

In [1]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# benchmark weights in N asset classes
q = np.array([0.25, 0.25, 0.25, 0.25])

In [130]:
# deviations from the benchmark (have to sum to zero!)
x = np.array([-0.05, 0.1 ,-0.05, 0])

In [131]:
# portfolio asset class weights
pf = q + x

In [132]:
E = np.array([0.03, 0.1, 0.05, 0.08])

In [133]:
# define correlation matrix, and vector of risk per asset class, which is then used to get the matrix..
rho = np.array([
        [1.0   ,0.1  ,0.4, -0.2]
        ,[0.1  ,1.0  ,0.5, 0.2]
        ,[0.4  ,0.5  ,1.0, 0.6]
        ,[-0.2 ,0.2  ,0.6, 1.0]
    ])
sigma = [0.03, 0.15, 0.1, 0.08]

In [134]:
def corr2cov(sigma, corr):
    return corr * np.outer(sigma, sigma)

In [135]:
# Variance-covariance matrix
V = corr2cov(sigma, rho)

In [136]:
V

array([[ 0.0009 ,  0.00045,  0.0012 , -0.00048],
       [ 0.00045,  0.0225 ,  0.0075 ,  0.0024 ],
       [ 0.0012 ,  0.0075 ,  0.01   ,  0.0048 ],
       [-0.00048,  0.0024 ,  0.0048 ,  0.0064 ]])

In [137]:
# Expected return on the benchmark
np.dot(q, E)

0.065000000000000002

In [138]:
# absolute volatility of benchmark
np.dot(np.dot(q, V), q)

0.0044712499999999995

In [139]:
# relative return
np.dot(x, E)

0.0060000000000000019

In [140]:
# Tracking error
np.dot(np.dot(x, V), x)

0.00017875000000000001

In [141]:
# actual portfolio return
np.dot(pf, E)

0.070999999999999994

In [142]:
# actual portfolio risk
np.dot(np.dot(pf, V), pf)

0.0056532500000000003

In [143]:
# alternative, expanded version of the portfolio risk calculation
np.dot(np.dot(q, V), q) + 2 * np.dot(np.dot(q, V), x) + np.dot(np.dot(x, V), x)

0.0056532499999999994

In [144]:
# Check that the benchmark weights sum to 1
np.dot(q, np.ones(q.size)) == 1

True

In [145]:
# Check that the deviations weights sum to 0
np.dot(x, np.ones(x.size)) == 0

True

In [146]:
def portfolio_return(weights, expected_returns):
    return np.dot(weights, expected_returns)

In [147]:
def portfolio_variance(weights, covar_matrix):
    return np.dot(np.dot(weights, covar_matrix), weights)

In [148]:
from scipy.optimize import minimize

In [180]:
cons = ({'type'  : 'eq','fun' : lambda x: np.dot(x, np.ones(x.size)) - 1}
        ,{'type' : 'ineq','fun' : lambda x: min(x)}
        ,{'type' : 'eq', 'fun' : lambda x: portfolio_return(x, E) - 0.05}
       )

In [181]:
myPort = minimize(portfolio_variance, [0.25, 0.25, 0.25, 0.25], args = V
                  ,method = "SLSQP", constraints = cons)

In [182]:
np.round(myPort['x'], 2)

array([ 0.62,  0.05, -0.  ,  0.33])

In [183]:
portfolio_return(myPort['x'], E)

0.050000000032102913

In [184]:
portfolio_variance(myPort['x'], V)

0.001008938519430256

In [96]:
# Try one version where we set up the constrained minimization in python
def optim_port1():
    pass

In [88]:
# then create one version of the code where we use the actual closed-form solutions which have been devised